In [ ]:
%pip install segment-geospatial

In [ ]:
import leafmap
from samgeo import tms_to_geotiff
from samgeo import SamGeo

In [ ]:
m = leafmap.Map(center=[-22.17615, -51.253043], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-51.2565, -22.1777, -51.2512, -22.175]

In [ ]:
image = "Image.tif"
tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

In [ ]:
# image = '/path/to/your/own/image.tif'

In [ ]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

In [ ]:
sam = SamGeo(
    model_type="vit_h",
    automatic=False,
    sam_kwargs=None,
)

In [ ]:
sam.set_image(image)

In [ ]:
m

In [ ]:
if m.user_rois is not None:
    boxes = m.user_rois
else:
    boxes = [
        [-51.2546, -22.1771, -51.2541, -22.1767],
        [-51.2538, -22.1764, -51.2535, -22.1761],
    ]

In [ ]:
sam.predict(boxes=boxes, point_crs="EPSG:4326", output="mask.tif", dtype="uint8")

In [ ]:
m.add_raster("mask.tif", cmap="viridis", nodata=0, layer_name="Mask")
m

In [ ]:
url = "https://opengeos.github.io/data/sam/tree_boxes.geojson"
geojson = "tree_boxes.geojson"
leafmap.download_file(url, geojson)

In [ ]:
m = leafmap.Map()
m.add_raster("Image.tif", layer_name="image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_vector(geojson, style=style, zoom_to_layer=True, layer_name="Bounding boxes")
m

In [ ]:
sam.predict(boxes=geojson, point_crs="EPSG:4326", output="mask2.tif", dtype="uint8")

In [ ]:
m.add_raster("mask2.tif", cmap="Greens", nodata=0, opacity=0.5, layer_name="Tree masks")
m

![](https://i.imgur.com/SpA2NV9.gif)